In [2]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [3]:
# Load data
df = pd.read_excel('data/Monthly Mastersheet.xlsx')

# Ensure date is datetime and set index
df['Month'] = pd.to_datetime(df['Month'])
df.set_index('Month', inplace=True)

In [ ]:
import pandas as pd
import statsmodels.api as sm
from itertools import combinations

# Clean column names
df.columns = df.columns.str.strip()

# Crypto assets
crypto_assets = ['Bitcoin', 'Tether', 'Litecoin', 'XRP', 'Ethereum', 'Dogecoin', 'Cardano', 'USD Coin']

# Compute Lagged Log Vol for each
for asset in crypto_assets:
    df[f'Lagged_LogVol_{asset}'] = df[asset].shift(1)

# Results storage
results = []

# Try all combinations of crypto assets (1 to all)
for k in range(1, len(crypto_assets) + 1):
    for combo in combinations(crypto_assets, k):
        lagged_cols = [f'Lagged_LogVol_{asset}' for asset in combo]
        temp_df = df[['GDP'] + lagged_cols].dropna()

        if len(temp_df) < 10:
            continue

        y = temp_df['GDP']
        X = sm.add_constant(temp_df[lagged_cols])

        model = sm.OLS(y, X).fit()

        results.append({
            'Assets': combo,
            'R^2': model.rsquared,
            'Adj. R^2': model.rsquared_adj,
            'N Obs': len(temp_df)
        })

# Display results sorted by Adjusted R²
results_df = pd.DataFrame(results).sort_values(by='Adj. R^2', ascending=False).reset_index(drop=True)

# Optional: format assets as a string
results_df['Assets'] = results_df['Assets'].apply(lambda x: ' + '.join(x))

print(results_df)


                                     Assets       R^2  Adj. R^2  N Obs
0               Bitcoin + Tether + Ethereum  0.543453  0.527148     88
1    Bitcoin + Tether + Ethereum + Dogecoin  0.548692  0.526942     88
2         Bitcoin + Tether + XRP + Ethereum  0.547732  0.525936     88
3              Tether + Ethereum + Dogecoin  0.540496  0.524085     88
4                   Tether + XRP + Ethereum  0.540321  0.523904     88
..                                      ...       ...       ...    ...
250                                 Bitcoin  0.140886  0.130896     88
251                      Bitcoin + Dogecoin  0.141449  0.121248     88
252                                     XRP  0.112965  0.102651     88
253                          XRP + Dogecoin  0.116514  0.095726     88
254                                Dogecoin  0.025203  0.013868     88

[255 rows x 4 columns]
